In [91]:
!pip install -q transformers==4.52.4
!pip install -q bitsandbytes==0.46.0
!pip install -q accelerate==1.7.0
!pip install -q langchain==0.3.25
!pip install -q langchainhub==0.1.21
!pip install -q langchain-chroma==0.2.4
!pip install -q langchain_experimental==0.3.4
!pip install -q langchain-community==0.3.24
!pip install -q langchain_huggingface==0.2.0
!pip install -q python-dotenv==1.1.0
!pip install -q pypdf
!pip install -q markitdown[pdf]

In [92]:
FILE_PATH = "./data-file/luat_anm_2018.pdf"

In [74]:
from langchain_community.document_loaders import PyPDFLoader

Loader = PyPDFLoader
loader = Loader(FILE_PATH)
document = loader.load()

In [ ]:
print(document)

In [93]:
md = MarkItDown(enable_plugins=False)
result = md.convert(FILE_PATH)

In [ ]:
print(result.text_content)

In [94]:
from langchain_core.documents import Document

document = Document(page_content=result.text_content, metadata={"source": "manual_string_creation"})

In [ ]:
print(document)

In [95]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name="bkai-foundation-models/vietnamese-bi-encoder"
)

In [96]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_splitter = SemanticChunker(
    embeddings=embedding,
    buffer_size=1,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95,
    min_chunk_size=500,
    add_start_index=True
)

In [97]:
docs = semantic_splitter.split_documents([document])

In [ ]:
for doc in docs:
    print(doc.page_content)
    print("=====================================================================")
    print("=====================================================================")
    print("=====================================================================")

In [98]:
from langchain_chroma import Chroma

vector_db = Chroma.from_documents(documents=docs, embedding=embedding)
retriever = vector_db.as_retriever()

In [ ]:
result = retriever.invoke("các hành vi nào bị nghiêm cấm?")
for r in result:
    print(r)
    print("=====================================================================")
    print("=====================================================================")
    print("=====================================================================")

In [ ]:
results_with_scores = vector_db.similarity_search_with_score("các hành vi nào bị nghiêm cấm?")

for doc, score in results_with_scores:
    print(f"Document: {doc.page_content}")
    print(f"Similarity Score: {score}")
    print("=====================================================================")
    print("=====================================================================")
    print("=====================================================================")

In [99]:
from transformers import BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [100]:
from transformers.models.auto.modeling_auto import AutoModelForCausalLM
from transformers import AutoTokenizer

MODEL_NAME = "lmsys/vicuna-7b-v1.5"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config,
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [101]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline

model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    device_map="auto"
)

llm = HuggingFacePipeline(
    pipeline=model_pipeline
)

Device set to use cuda:0


In [102]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [103]:
USER_QUESTION = "Hiệu lực thi hành của luật an ninh mạng?"
output = rag_chain.invoke(USER_QUESTION)
print(output)
answer = output.split("Answer:")[1].strip()
print(answer)

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: Hiệu lực thi hành của luật an ninh mạng? 
Context: 2. Kịp thời cung cấp thông tin liên quan đến bảo vệ an ninh mạng, nguy cơ đe dọa an ninh mạng,
hành vi xâm phạm an ninh mạng cho cơ quan có thẩm quyền, lực lượng bảo vệ an ninh mạng. 3. Thực hiện yêu cầu và hướng dẫn của cơ quan có thẩm quyền trong bảo vệ an ninh mạng; giúp
đỡ, tạo điều kiện cho cơ quan, tổ chức và người có trách nhiệm tiến hành các biện pháp bảo vệ
an ninh mạng. Chương VII

ĐIỀU KHOẢN THI HÀNH

Điều 43. Hiệu lực thi hành

1. Luật này có hiệu lực thi hành từ ngày 01 tháng 01 năm 2019.

2. Kịp thời cung cấp thông tin liên quan đến bảo vệ an ninh mạng, nguy cơ đe dọa an ninh mạng,
hành vi xâm phạm an ninh mạng cho cơ quan có thẩm quyền, lực lượng bảo vệ an ninh mạng. 3. 

In [106]:
USER_QUESTION = "Cổng kết nối mạng quốc tế là gì?"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split("Answer:")[1].strip()
print(answer)

Cổng kết nối mạng quốc tế là nơi diễn ra hoạt động chuyển nhận tín hiệu mạng qua lại giữa Việt Nam và các quốc gia, vùng lãnh thổ khác.


In [107]:
USER_QUESTION = "Tóm tắt các chính sách của nhà nước về an ninh mạng?"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split("Answer:")[1].strip()
print(answer)


OutOfMemoryError: CUDA out of memory. Tried to allocate 230.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 92.12 MiB is free. Process 8302 has 14.65 GiB memory in use. Of the allocated memory 12.93 GiB is allocated by PyTorch, and 1.59 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)